In [1]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
import praw
import pandas as pd

In [2]:
import praw
from datetime import datetime, timedelta
from dotenv import dotenv_values
import pandas as pd

# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keywords(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    # Define the date range for March and April (2024)
    today = datetime(2024, 4, 30, 23, 59, 59)  # End of April
    start_date = datetime(2024, 3, 1)  # Start of March
    
    # Iterate through each keyword
    for keyword in keywords:
        print(f"Fetching posts for keyword: {keyword}")
        for post in subreddit.search(keyword, time_filter='all', limit=limit):
            post_date = datetime.utcfromtimestamp(post.created)
            
            # Check if the post is within the desired time range
            if start_date <= post_date <= today:
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keyword': keyword
                    })
    
    return data

subreddit_name = "stocks"
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

# Fetch posts from March and April 2024
data = fetch_posts_about_keywords(subreddit_name, keywords, limit=500000)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("march_april_reddit_comments.csv", index=False)

# Display the date range of the fetched posts
if not df.empty:
    date_range = df['post_created'].agg(['min', 'max'])
    print(f"Date range: from {date_range['min']} to {date_range['max']}")
else:
    print("No data found for the specified range.")


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


Fetching posts for keyword: Uber


/tmp/ipykernel_38906/4009646819.py:28: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(post.created)


Fetching posts for keyword: youtube
Fetching posts for keyword: meta
Fetching posts for keyword: apple
Fetching posts for keyword: nvidia
Fetching posts for keyword: microsoft
Fetching posts for keyword: amazon
Fetching posts for keyword: Saudi Arabian Oil Co
Fetching posts for keyword: intel
Fetching posts for keyword: tesla
Fetching posts for keyword: MARA Holdings
Date range: from 2024-03-01 03:45:26 to 2024-04-30 20:07:29


In [3]:
if not df.empty:
    date_range = df['post_created'].agg(['min', 'max'])
    print(f"Date range: from {date_range['min']} to {date_range['max']}")
else:
    print("No data found for the specified range.")

Date range: from 2024-03-01 03:45:26 to 2024-04-30 20:07:29


In [4]:
df.describe()

,post_score,post_num_comments,post_created,comment_score
count,23989.000000,23989.000000,23989,23989.000000
mean,1343.609113,428.184751,2024-03-31 08:08:52.474926080,13.364542
min,0.000000,4.000000,2024-03-01 03:45:26,-103.000000
25%,440.000000,225.000000,2024-03-18 16:10:05,1.000000
50%,937.000000,352.000000,2024-03-30 18:49:49,2.000000
75%,1687.000000,637.000000,2024-04-15 20:15:02,7.000000
max,9017.000000,1690.000000,2024-04-30 20:07:29,5827.000000
std,1502.427084,296.515615,NaN,73.317834


In [5]:
df.columns

Index(['post_title', 'post_score', 'post_id', 'post_num_comments',
       'post_created', 'post_body', 'comment_id', 'comment_body',
       'comment_score', 'matched_keyword'],
      dtype='object')